In [1]:
import math
import pylab
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F

seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)

torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)



class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1

In [2]:
from collections import OrderedDict
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.L = 128
        self.D = 64
        self.K = 1
        self.feature_size = 24
        self.shared_layer_size = 128

        self.fc = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=(2,1),padding=1,stride=1),
            nn.ReLU(),
            nn.Conv1d(128, 64, kernel_size=(2,1),padding=1,stride=1),
            nn.ReLU()
          )
        self.fc2 = nn.Sequential(
            nn.Linear(37632, 512),
            nn.BatchNorm1d(512,affine=False),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        
        Y_prob=self.fc(x)
        Y_prob=self.fc2(Y_prob.reshape(Y_prob.size(0),-1))
        
        return Y_prob

In [3]:
from torch.optim.lr_scheduler import _LRScheduler
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [4]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [5]:
xTrain=np.load("../extracted_features/0/GRP-3/input_features_train_grp3.npy")
yTrain=np.load("../extracted_features/0/GRP-3/labels_train_grp3.npy")

xTest=np.load("../extracted_features/0/GRP-3/input_features_test_grp3.npy")
yTest=np.load("../extracted_features/0/GRP-3/labels_test_grp3.npy")


xVal=np.load("../extracted_features/0/GRP-3/input_features_val_grp3.npy")
yVal=np.load("../extracted_features/0/GRP-3/labels_val_grp3.npy")

train_features=np.load("../extracted_features/bl/0/GRP-3/input_features_train_grp3.npy")
labels_train=np.load("../extracted_features/bl/0/GRP-3/labels_train_grp3.npy")

test_features=np.load("../extracted_features/bl/0/GRP-3/input_features_test_grp3.npy")
labels_test=np.load("../extracted_features/bl/0/GRP-3/labels_test_grp3.npy")

val_features=np.load("../extracted_features/bl/0/GRP-3/input_features_val_grp3.npy")
labels_val=np.load("../extracted_features/bl/0/GRP-3/labels_val_grp3.npy")


In [6]:
print(labels_train.shape,labels_train)

(497,) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [7]:
print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)

(512, 19, 24) (512,)
(122, 19, 24) (122,)
(118, 19, 24) (118,)


In [8]:
print(train_features.shape,labels_train.shape)
print(test_features.shape,labels_test.shape)
print(val_features.shape,labels_val.shape)

(497, 19, 24) (497,)
(119, 19, 24) (119,)
(110, 19, 24) (110,)


In [9]:
train_features=train_features.reshape(len(train_features),1,19,24)
test_features=test_features.reshape(len(test_features),1,19,24)
val_features=val_features.reshape(len(val_features),1,19,24)

xTrain = xTrain.reshape(len(xTrain),1,19,24)
xTest = xTest.reshape(len(xTest),1,19,24)
xVal = xVal.reshape(len(xVal),1,19,24)

In [10]:
# xTrain=np.append(train_features,xTrain,axis=0)
# xTest=np.append(test_features,xTest,axis=0)
# xVal=np.append(val_features,xVal,axis=0)
# yTrain=np.append(labels_train,yTrain,axis=0)
# yTest=np.append(labels_test,yTest,axis=0)
# yVal=np.append(labels_val,yVal,axis=0)

In [11]:
batch_size=20

train_data = Dataset(xTrain, yTrain)
test_data=Dataset(xTest,yTest)
val_data=Dataset(xVal,yVal)

# train_data = Dataset(train_features, labels_train)
# test_data=Dataset(test_features,labels_test)
# val_data=Dataset(val_features,labels_val)

train_data_loader = DataLoader(train_data,worker_init_fn=seed_worker, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(test_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(val_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

In [12]:
for batch in val_data_loader:
    print(batch[1].shape)
    break
print(len(val_data_loader))

torch.Size([20])
7


In [13]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 100
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
model.apply(init_weights)
loss_func = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [14]:
from torchsummary import summary
summary(model.cuda(),(1,19,24))
# torch.save(model, "modelarchitect_final.pt")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1          [-1, 128, 20, 26]             384
              ReLU-2          [-1, 128, 20, 26]               0
            Conv1d-3           [-1, 64, 21, 28]          16,448
              ReLU-4           [-1, 64, 21, 28]               0
            Linear-5                  [-1, 512]      19,268,096
       BatchNorm1d-6                  [-1, 512]               0
              ReLU-7                  [-1, 512]               0
            Linear-8                  [-1, 128]          65,664
              ReLU-9                  [-1, 128]               0
           Linear-10                   [-1, 64]           8,256
             ReLU-11                   [-1, 64]               0
           Linear-12                    [-1, 1]              65
          Sigmoid-13                    [-1, 1]               0
Total params: 19,358,913
Trainable para

In [15]:
from sklearn.metrics import accuracy_score
best_acc1 = 0
modelname=[]
truth=[]
preds=[]
model.train()
for it in range(epoch+1):
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=accuracy_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))
    
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()    
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=accuracy_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        print('Best validation accuracy1 ', best_acc1)

EPOCH  0
Train : Loss: 0.0344, Train acc1 : 0.5977
Val : Loss: 0.0296, Val acc1 : 0.6475
Best validation accuracy1  0.6475409836065574
EPOCH  1
Train : Loss: 0.0354, Train acc1 : 0.6167
Val : Loss: 0.0292, Val acc1 : 0.6967
Best validation accuracy1  0.6967213114754098
EPOCH  2
Train : Loss: 0.0352, Train acc1 : 0.5915
Val : Loss: 0.0290, Val acc1 : 0.7541
Best validation accuracy1  0.7540983606557377
EPOCH  3
Train : Loss: 0.0304, Train acc1 : 0.6546
Val : Loss: 0.0282, Val acc1 : 0.6967
EPOCH  4
Train : Loss: 0.0332, Train acc1 : 0.6073
Val : Loss: 0.0289, Val acc1 : 0.7787
Best validation accuracy1  0.7786885245901639
EPOCH  5
Train : Loss: 0.0287, Train acc1 : 0.6909
Val : Loss: 0.0296, Val acc1 : 0.7295
EPOCH  6
Train : Loss: 0.0308, Train acc1 : 0.6562
Val : Loss: 0.0357, Val acc1 : 0.6967
EPOCH  7
Train : Loss: 0.0273, Train acc1 : 0.7129
Val : Loss: 0.0285, Val acc1 : 0.7213
EPOCH  8
Train : Loss: 0.0286, Train acc1 : 0.7066
Val : Loss: 0.0323, Val acc1 : 0.6066
EPOCH  9
Train 

EPOCH  90
Train : Loss: 0.0213, Train acc1 : 0.8360
Val : Loss: 0.0712, Val acc1 : 0.5082
EPOCH  91
Train : Loss: 0.0028, Train acc1 : 0.9022
Val : Loss: 0.0766, Val acc1 : 0.5164
EPOCH  92
Train : Loss: 0.0018, Train acc1 : 0.9054
Val : Loss: 0.1153, Val acc1 : 0.5328
EPOCH  93
Train : Loss: 0.0009, Train acc1 : 0.9085
Val : Loss: 0.0965, Val acc1 : 0.5410
EPOCH  94
Train : Loss: 0.0010, Train acc1 : 0.9101
Val : Loss: 0.1249, Val acc1 : 0.5410
EPOCH  95
Train : Loss: 0.0006, Train acc1 : 0.9117
Val : Loss: 0.0945, Val acc1 : 0.5492
EPOCH  96
Train : Loss: 0.0008, Train acc1 : 0.9117
Val : Loss: 0.1192, Val acc1 : 0.5246
EPOCH  97
Train : Loss: 0.0004, Train acc1 : 0.9085
Val : Loss: 0.1229, Val acc1 : 0.5410
EPOCH  98
Train : Loss: 0.0004, Train acc1 : 0.9117
Val : Loss: 0.1075, Val acc1 : 0.5328
EPOCH  99
Train : Loss: 0.0003, Train acc1 : 0.9101
Val : Loss: 0.1281, Val acc1 : 0.5574
EPOCH  100
Train : Loss: 0.0005, Train acc1 : 0.9148
Val : Loss: 0.1389, Val acc1 : 0.5410


In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.58 0.54 0.58 0.50 0.58 0.65


In [18]:
xTest=np.load("../extracted_features/0/win-1/GRP-3/input_features_val_grp3.npy")
yTest=np.load("../extracted_features/0/win-1/GRP-3/labels_val_grp3.npy")

xTest = xTest.reshape(len(xTest),1,19,24)

testdataset = Dataset(xTest,yTest)

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.49 0.42 0.48 0.38 0.49 0.60
